A `Process` can be used like an event.

Once you yield an event, you are resumed once the process has finished.

In [7]:
import simpy

class Car:
    def __init__(self, env: simpy.Environment):
        self.env = env
        # start the run process when an instance is created
        self.action = env.process(
            generator=self.run(), # env.process: create a new Process instance for generator
        )

    def run(self):
        while True:
            print(f"Start parking and charging at {self.env.now}")
            charge_duration = 5
            # yield the process that process() returns to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # Return here once charge process has finished, and we can drive again
            print(f"Start driving at {self.env.now}")
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration) # return a Timeout event


env = simpy.Environment()
car = Car(env)

env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


We want to permit interrupts to the charging process. We can interrupt a `Process` by calling the `interrupt` method.

In [11]:
import simpy

class Car:
    def __init__(self, env: simpy.Environment):
        self.env = env
        # start the run process when an instance is created
        self.action = env.process(
            generator=self.run(), # env.process: create a new Process instance for generator
        )

    def run(self):
        while True:
            print(f"Start parking and charging at {self.env.now}")
            charge_duration = 5

            try:
                # yield the process that process() returns to wait for it to finish
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # if interrupted, stop charging and switch back to driving
                print(f"Charging was interrupted at {self.env.now}. Hope the battery is full enough...")

            # Return here once charge process has finished, and we can drive again
            print(f"Start driving at {self.env.now}")
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration) # return a Timeout event


def driver(env: simpy.Environment, car: Car):
    """Process"""
    yield env.timeout(3)
    car.action.interrupt()


env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))

env.run(until=15)

Start parking and charging at 0
Charging was interrupted at 3. Hope the battery is full enough...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


: 